In [35]:
!pip install scikit-optimize
!pip install -U pymoo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [36]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import cifar10
from skopt import gp_minimize
import matplotlib.pyplot as plt

In [39]:
# Carregar os dados
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

train_mask = np.logical_or(y_train == 0, y_train == 1).flatten()
test_mask = np.logical_or(y_test == 0, y_test == 1).flatten()
x_train, y_train = x_train[train_mask], y_train[train_mask]
x_test, y_test = x_test[test_mask], y_test[test_mask]

# Normalizar os dados
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
y_train = keras.utils.to_categorical(y_train, 2)
y_test = keras.utils.to_categorical(y_test, 2)

In [46]:
def build_cnn_model():
    # Criar o modelo da rede neural convolucional
    model = keras.Sequential([
        layers.Input(shape=(32, 32, 3)),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation="relu"),
        layers.Dense(2, activation="softmax")
    ])

    return model

In [41]:
def chebyshev_loss(y_true, y_pred, model, lambda1, lambda2):
    # Função de perda Categorical Crossentropy
    ce_loss = keras.losses.CategoricalCrossentropy()(y_true, y_pred)

    # Função de regularização L2
    l2_loss = sum([tf.nn.l2_loss(layer.kernel) for layer in model.layers if 'Conv2D' in str(layer) or 'Dense' in str(layer)])

    # Combinar as funções de perda com os pesos de Chebyshev
    combined_loss = lambda1 * ce_loss + lambda2 * l2_loss

    return combined_loss

In [42]:
def objective(params):
    lambda1, lambda2 = params
    
    # Construir o modelo da CNN
    model = build_cnn_model()

    # Compilar o modelo com a função de perda personalizada e otimizador
    model.compile(loss=lambda y_true, y_pred: chebyshev_loss(y_true, y_pred, model, lambda1, lambda2), optimizer=keras.optimizers.Adam(), metrics=["accuracy"])

    # Treinar o modelo
    model.fit(x_train, y_train, batch_size=128, epochs=2, validation_split=0.1, verbose=1)

    # Avaliar o desempenho do modelo
    score = model.evaluate(x_test, y_test, verbose=0)
    
    return score[0]

In [47]:
space = [(0.0001, 0.01, "uniform"), (0.0001, 0.01, "uniform")]

result = gp_minimize(func=objective, dimensions=space, n_calls=10, random_state=42, verbose=1)

#Imprimir os melhores hiperparâmetros encontrados
best_lambda1, best_lambda2 = result.x
print(f"Melhores hiperparâmetros encontrados: lambda1 = {best_lambda1}, lambda2 = {best_lambda2}")

Iteration No: 1 started. Evaluating function at random point.
Epoch 1/2
71/71 [==============================] - 9s 111ms/step - loss: 0.0714 - accuracy: 0.6180 - val_loss: 0.0094 - val_accuracy: 0.7140
Epoch 2/2
71/71 [==============================] - 7s 95ms/step - loss: 0.0072 - accuracy: 0.6842 - val_loss: 0.0061 - val_accuracy: 0.7030
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 16.4398
Function value obtained: 0.0061
Current minimum: 0.0061
Iteration No: 2 started. Evaluating function at random point.
Epoch 1/2
71/71 [==============================] - 8s 97ms/step - loss: 0.2072 - accuracy: 0.5317 - val_loss: 0.0167 - val_accuracy: 0.4980
Epoch 2/2
71/71 [==============================] - 8s 110ms/step - loss: 0.0099 - accuracy: 0.5002 - val_loss: 0.0069 - val_accuracy: 0.5020
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 16.4696
Function value obtained: 0.0069
Current minimum: 0.0061
Iteration No: 3 started. Evaluating function at ra

In [48]:
#Construir e treinar o melhor base
base_model = build_cnn_model()
base_model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(), metrics=["accuracy"])
base_model.fit(x_train, y_train, batch_size=128, epochs=2, validation_split=0.1, verbose=1)

#Avaliar o melhor base
score = base_model.evaluate(x_test, y_test, verbose=1)
print(f"Test loss: {score[0]}, Test accuracy: {score[1]}")

Epoch 1/2
71/71 [==============================] - 9s 112ms/step - loss: 0.4594 - accuracy: 0.7807 - val_loss: 0.3383 - val_accuracy: 0.8610
Epoch 2/2
63/63 [==============================] - 1s 10ms/step - loss: 0.2616 - accuracy: 0.8870
Test loss: 0.2615981101989746, Test accuracy: 0.8870000243186951


In [51]:
#Construir e treinar o melhor modelo
best_model = build_cnn_model()
best_model.compile(loss=lambda y_true, y_pred: chebyshev_loss(y_true, y_pred, best_model, best_lambda1, best_lambda2),
optimizer=keras.optimizers.Adam(), metrics=["accuracy"])
best_model.fit(x_train, y_train, batch_size=128, epochs=2, validation_split=0.1, verbose=1)

#Avaliar o melhor modelo
score = best_model.evaluate(x_test, y_test, verbose=1)
print(f"Test loss: {score[0]}, Test accuracy: {score[1]}")

Epoch 1/2
71/71 [==============================] - 13s 178ms/step - loss: 0.1778 - accuracy: 0.4947 - val_loss: 0.0100 - val_accuracy: 0.4980
Epoch 2/2
63/63 [==============================] - 1s 8ms/step - loss: 0.0015 - accuracy: 0.5000
Test loss: 0.0014701948966830969, Test accuracy: 0.5
